<a href="https://colab.research.google.com/github/jose261116/meu-primeiro-contato/blob/main/projeto_classificacao_pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from google.colab import drive

In [40]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Em 'local_data' deverá passar o caminho do dataset em zip

In [41]:
#!unzip '/content/drive/MyDrive/chest_xray'

In [42]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import keras.utils as image
from keras.utils import load_img, img_to_array
import numpy as np
from keras.preprocessing import image

In [43]:
#Definição da rede neural
classificador = Sequential()

#Criando a camada de convolucao de 32 filtros, e dimensoes 3x3, altura e largura da imagem
classificador.add(Conv2D(32, (3,3), input_shape = (64, 64, 3), activation = 'relu'))

#Acelera o processamento com os dados em escala entre 0 e 1
classificador.add(BatchNormalization())

#Uma matrix de 4 pixels
classificador.add(MaxPooling2D(pool_size = (2,2)))

In [44]:



classificador.add(Conv2D(32, (3,3), input_shape = (64, 64, 3), activation = 'relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size = (2,2)))

In [45]:
# Transforma a matrix em um vetor
classificador.add(Flatten())

# Rede neural densa
classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dropout(0.2))

classificador.add(Dense(units = 128, activation = 'relu'))

# Zerando 20% das entradas
classificador.add(Dropout(0.2))
classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dropout(0.2))

# Camada de saida com classificação binaria
classificador.add(Dense(units = 1, activation = 'sigmoid'))

In [46]:
classificador.compile (optimizer = 'adam', loss = 'binary_crossentropy',
                      metrics = ['accuracy'])

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
# Gerando imagens aleatorias aproveitando os dados existentes
gerador_treinamento = ImageDataGenerator(rescale = 1./255,
                                         rotation_range = 7,
                                         horizontal_flip = True,
                                         shear_range = 0.2,
                                         height_shift_range = 0.07,
                                         zoom_range = 0.2)

### Em 'pasta_de_treino' e 'pasta_de_teste' deverá passar o caminho da pasta de treino e de teste da sua pasta contendo o dataset.

In [50]:
gerador_teste = ImageDataGenerator(rescale = 1./255)

# Definindo a base de dados
base_treinamento = gerador_treinamento.flow_from_directory('/content/drive/MyDrive/chest_xray/chest_xray/train',
                                                           target_size = (64, 64),
                                                           batch_size = 32,
                                                           class_mode = 'binary')


base_teste = gerador_teste.flow_from_directory('/content/drive/MyDrive/chest_xray/chest_xray/test',
                                               target_size = (64, 64),
                                               batch_size = 32,
                                               class_mode = 'binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


### Em defina o passo para o treino. Recomendável setar o tamanho do dataset de treino e para validation_teps o tamanho do dataset de teste

In [51]:

















































# Treinando a rede neural
classificador.fit_generator(base_treinamento, steps_per_epoch = 4000 / 32,
                            epochs = 50, validation_data = base_teste,
                            validation_steps = 1000 / 32)

<ipython-input-51-9ba43c5a2fc2>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classificador.fit_generator(base_treinamento, steps_per_epoch = 4000 / 32,


Epoch 1/50
125/125 [==============================] - ETA: 0s - loss: 0.3209 - accuracy: 0.8677

125/125 [==============================] - 793s 6s/step - loss: 0.3209 - accuracy: 0.8677 - val_loss: 1.4863 - val_accuracy: 0.6250
Epoch 2/50
125/125 [==============================] - 189s 2s/step - loss: 0.2472 - accuracy: 0.8985
Epoch 3/50
125/125 [==============================] - 88s 707ms/step - loss: 0.1886 - accuracy: 0.9260
Epoch 4/50
125/125 [==============================] - 67s 536ms/step - loss: 0.2058 - accuracy: 0.9165
Epoch 5/50
125/125 [==============================] - 60s 483ms/step - loss: 0.1793 - accuracy: 0.9342
Epoch 6/50
125/125 [==============================] - 59s 475ms/step - loss: 0.1730 - accuracy: 0.9340
Epoch 7/50
125/125 [==============================] - 58s 467ms/step - loss: 0.1750 - accuracy: 0.9317
Epoch 8/50
125/125 [==============================] - 58s 467ms/step - loss: 0.1545 - accuracy: 0.9427
Epoch 9/50
125/125 [==============================] - 59s 472ms/step - loss: 0.1609 - accuracy: 0.9377
Epoch 10/50
125/125 [=========================

In [53]:
# Carregando uma imagem para classificação
imagem_teste = load_img('/content/drive/MyDrive/chest_xray/train/NORMAL/IM-0131-0001.jpeg',
                              target_size = (64,64))

imagem_teste = img_to_array(imagem_teste)
imagem_teste /= 255
imagem_teste = np.expand_dims(imagem_teste, axis = 0)

#Realizando a previsão
previsao = classificador.predict(imagem_teste)
base_treinamento.class_indices
previsao

1/1 [==============================] - 0s 263ms/step


array([[0.05601394]], dtype=float32)

In [54]:
previsao, base_treinamento.class_indices

(array([[0.05601394]], dtype=float32), {'NORMAL': 0, 'PNEUMONIA': 1})

In [55]:
classificador.save('pneumonia.h5')